In [21]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
import json
from Tree import Tree
import os
# os.environ["PATH"] += os.pathsep + 'C:/Users/krzys/AppData/Local/Programs/Python/Python38/Libsite-packages/graphviz'
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

In [22]:
with open('data/iron1.json') as f:
    data_json = json.load(f)
champions_names = set()
traits_names = set()
for game in data_json:
    for i in range(0,len(game),4):
        champions = game[i]
        traits = game[i+1]
        dps = game[i+2]
        for champion in champions:
            champions_names.add(champion[0])
        for tr in traits:
            traits_names.add(tr[0])
                
champions_names = list(champions_names)
traits_names = list(traits_names)

In [23]:
champions_names = [str(name)[5:] for name in champions_names]
# champions_names

In [24]:
traits_names = [str(name)[5:] if 'Set2_' in str(name) else name for name in traits_names]
# traits_names

In [25]:
def prepare_data(data_json, champions_names, traits_names, division):
    df = pd.DataFrame(columns=champions_names+traits_names+['dps']+['won']+['lost']+['target'], dtype=np.float32)
    for i, game in enumerate(data_json):
        # jako że elementy w game nie sa po kolei to trzeba przez wszystkie przeiterowac i sprawdzic, jesli dany z nich to 'placement'
        # to wtedy jeżeli placement ==1 to ==> k-1 to win dpsy, k-2 to win traity a k-3 to win champy
        # jeżeli placement == 8 to ==> analogicznie
        for k in range(len(game)):
            if k % 4 == 3:
                if game[k] == 1:
                    win_index = k
                elif game[k] == 8:
                    lose_index = k
        # WON
        
        won_champions = game[win_index-3]
        won_traits = game[win_index-2]
        won_dps = game[win_index-1]

        won_champions = [c[0] for c in won_champions]    
        won_traits_nr = [np.round(c[2]/c[3]*100,0) for c in won_traits]
        won_traits = [c[0] for c in won_traits]

        won_traits = [str(name)[5:] if 'Set2_' in str(name) else name for name in won_traits]
        won_champions = [str(name)[5:] for name in won_champions]
        
        won_row = np.empty(len(champions_names)+len(traits_names)+4, dtype=np.int)
        won_row.fill(0)
    
        # LOST
    
        lost_champions = game[lose_index-3]
        lost_traits = game[lose_index-2]
        lost_dps = game[lose_index-1]

        lost_row = np.empty(len(champions_names)+len(traits_names)+4, dtype=np.int)
        lost_row.fill(0)

        lost_champions = [c[0] for c in lost_champions]    
        lost_traits_nr = [np.round(c[2]/c[3]*100,0) for c in lost_traits]
        lost_traits = [c[0] for c in lost_traits]

        lost_traits = [str(name)[5:] if 'Set2_' in str(name) else name for name in lost_traits]
        lost_champions = [str(name)[5:] for name in lost_champions]

        

        for j, champion in enumerate(champions_names):
            if champion in won_champions:
                won_row[j] = 1
            if champion in lost_champions:
                lost_row[j] = 1

        for j, trait in enumerate(traits_names):
            if trait in won_traits:
                won_row[len(champions_names)+j] = won_traits_nr[won_traits.index(trait)]
            if trait in lost_traits:
                lost_row[len(champions_names)+j] = lost_traits_nr[lost_traits.index(trait)]

        won_row[len(champions_names)+len(traits_names)] = won_dps
        lost_row[len(champions_names)+len(traits_names)] = lost_dps

        won_row[len(champions_names)+len(traits_names)+1] = 1
        won_row[len(champions_names)+len(traits_names)+2] = 0

        lost_row[len(champions_names)+len(traits_names)+1] = 0
        lost_row[len(champions_names)+len(traits_names)+2] = 1

        won_row[len(champions_names)+len(traits_names)+3] = 0
        lost_row[len(champions_names)+len(traits_names)+3] = 0


        df.loc[2*i]   = won_row
        df.loc[2*i+1] = lost_row
    df['target'] = division
    return df

In [26]:
iron1_df = prepare_data(data_json, champions_names, traits_names, 'iron1')
iron1_df.sample(10)

,Sion,Kindred,Annie,Soraka,LeBlanc,LuxMetal,Olaf,LuxLight,LuxElectric,Sivir,...,Wind,Summoner,Shadow,Desert,Blademaster,Metal,dps,won,lost,target
295,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,22.0,0.0,1.0,iron1
742,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,176.0,1.0,0.0,iron1
703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,iron1
459,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,50.0,0.0,0.0,0.0,0.0,30.0,0.0,1.0,iron1
119,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,1.0,iron1
664,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,180.0,1.0,0.0,iron1
684,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,208.0,1.0,0.0,iron1
745,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,53.0,0.0,1.0,iron1
357,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,50.0,0.0,0.0,0.0,52.0,0.0,1.0,iron1
223,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,49.0,0.0,1.0,iron1


In [27]:
with open('data/bronze1.json') as f:
    bronze1_data = json.load(f)
    
bronze1_df = prepare_data(bronze1_data, champions_names, traits_names, 'bronze1')
bronze1_df.sample(5)

,Sion,Kindred,Annie,Soraka,LeBlanc,LuxMetal,Olaf,LuxLight,LuxElectric,Sivir,...,Wind,Summoner,Shadow,Desert,Blademaster,Metal,dps,won,lost,target
378,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,186.0,1.0,0.0,bronze1
317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,50.0,0.0,0.0,0.0,0.0,14.0,0.0,1.0,bronze1
921,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,1.0,bronze1
49,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,1.0,bronze1
257,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,50.0,0.0,0.0,27.0,0.0,1.0,bronze1


In [28]:
with open('data/silver1.json') as f:
    silver1_data = json.load(f)
    
silver1_df = prepare_data(silver1_data, champions_names, traits_names, 'silver1')
silver1_df.sample(5)

,Sion,Kindred,Annie,Soraka,LeBlanc,LuxMetal,Olaf,LuxLight,LuxElectric,Sivir,...,Wind,Summoner,Shadow,Desert,Blademaster,Metal,dps,won,lost,target
372,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,50.0,50.0,0.0,0.0,0.0,175.0,1.0,0.0,silver1
901,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,1.0,silver1
197,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0,silver1
156,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,205.0,1.0,0.0,silver1
242,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,220.0,1.0,0.0,silver1


In [29]:
with open('data/gold1.json') as f:
    gold1_data = json.load(f)
    
gold1_df = prepare_data(gold1_data, champions_names, traits_names, 'gold1')
gold1_df.sample(5)

,Sion,Kindred,Annie,Soraka,LeBlanc,LuxMetal,Olaf,LuxLight,LuxElectric,Sivir,...,Wind,Summoner,Shadow,Desert,Blademaster,Metal,dps,won,lost,target
729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,38.0,0.0,1.0,gold1
571,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,gold1
124,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,50.0,0.0,0.0,144.0,1.0,0.0,gold1
266,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,50.0,0.0,0.0,0.0,0.0,185.0,1.0,0.0,gold1
213,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,1.0,gold1


In [30]:
with open('data/plat1.json') as f:
    plat1_data = json.load(f)
    
plat1_df = prepare_data(plat1_data, champions_names, traits_names, 'plat1')
plat1_df.sample(5)

,Sion,Kindred,Annie,Soraka,LeBlanc,LuxMetal,Olaf,LuxLight,LuxElectric,Sivir,...,Wind,Summoner,Shadow,Desert,Blademaster,Metal,dps,won,lost,target
525,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,33.0,24.0,0.0,1.0,plat1
101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,35.0,0.0,1.0,plat1
873,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,70.0,0.0,1.0,plat1
156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,33.0,0.0,0.0,50.0,67.0,0.0,179.0,1.0,0.0,plat1
181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,50.0,33.0,0.0,30.0,0.0,1.0,plat1


In [54]:
multidiv_df = plat1_df.append(gold1_df).append(silver1_df).append(bronze1_df).append(iron1_df)
multidiv_df.reset_index(inplace=True)
multidiv_df

,index,Sion,Kindred,Annie,Soraka,LeBlanc,LuxMetal,Olaf,LuxLight,LuxElectric,...,Wind,Summoner,Shadow,Desert,Blademaster,Metal,dps,won,lost,target
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,100.0,0.0,0.0,0.0,0.0,0.0,186.0,1.0,0.0,plat1
1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,50.0,0.0,0.0,0.0,26.0,0.0,1.0,plat1
2,2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,50.0,0.0,0.0,169.0,1.0,0.0,plat1
3,3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,1.0,plat1
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,223.0,1.0,0.0,plat1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4843,883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,1.0,iron1
4844,884,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,33.0,0.0,185.0,1.0,0.0,iron1
4845,885,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,iron1
4846,886,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,201.0,1.0,0.0,iron1


In [56]:
train = multidiv_df.sample(frac=0.8)
test  = multidiv_df.drop(train.index)
# X_train = train.drop('division', axis=1)
# X_test  =  test.drop('division', axis=1)
# Y_train = train['division']
# Y_test  =  test['division']
X_train = train.drop('target', axis=1)
X_test  =  test.drop('target', axis=1)
Y_train = train.target
Y_test  =  test.target 

In [57]:
multidiv_tree = Tree(train)

In [60]:
multidiv_Y_predict = [multidiv_tree.classify(X_test.iloc[i]) for i in range(len(X_test))]
# multidiv_tree
# multidiv_Y_predict

In [61]:
multidiv_Y_predict = [multidiv_tree.classify(X_test.iloc[i]) for i in range(len(X_test))]
multidiv_Y_predict = pd.Series(multidiv_Y_predict)
multidiv_Y_predict.index = Y_test.index
multidiv_good = multidiv_Y_predict == Y_test
multidiv_good.value_counts()

False    758
True     212
dtype: int64